# 핵심 라이브러리 설치
- 설치 후 세션 재시작 필요

In [ ]:
!pip install -q transformers accelerate bitsandbytes peft wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 37.2 MB/s eta 0:00:00


In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델 정의

In [ ]:
import os, re, math, random
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import torch
from typing import Any
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from tqdm import tqdm
import os
import torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:256"

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

AMP_DTYPE = torch.bfloat16

Image.MAX_IMAGE_PIXELS = None
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
MODEL_ID = "Qwen/Qwen3-VL-8B-Instruct"
IMAGE_SIZE = 448
MAX_NEW_TOKENS = 8
SEED = 1004
random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# 데이터 로드

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/train_hint_tasktype.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/test_hint_tasktype.csv")

# 모델 로드

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)

base_model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

base_model = prepare_model_for_kbit_training(base_model)
base_model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.72G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

trainable params: 43,646,976 || all params: 8,810,770,672 || trainable%: 0.4954


# Prompt 정의
- task에 대한 hint와
- OCR 결과를 hint로 제공

In [ ]:
SYSTEM_INSTRUCT = (
    "너는 시각적 질문에 답하는 인공지능 어시스턴트야."
    "작업을 생각하고 질문과 이미지에서 추출된 단어(힌트)를 함께 생각해."
    "보기 a, b, c, d 중에서 질문에 정확히 맞는 하나의 정답을 선택해."
    "정답의 알파벳만 출력해. (예: a, b, c 또는 d)"
    "최종 출력은 반드시 a, b, c, d 중 하나의 소문자 한 글자만 작성해. 설명 금지."
)

def build_mc_prompt(question, a, b, c, d, word, task):
    return (
        f"작업: {task}\n\n"
        f"질문: {question}\n\n"
        f"이미지에서 추출된 단어(힌트): {word}\n\n"
        f"보기:\n(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "이미지를 관찰한 후, 이미지에서 추출된 단어를 고려하여 질문에 정확히 맞는 보기를 선택해. "
        "설명은 필요없어. 정답은 반드시 소문자 알파벳 a, b, c, d 중 하나만 출력해. (a, b, c, 또는 d)"
    )

In [ ]:
class VQAMCDataset(Dataset):
    def __init__(self, df, processor, train=True):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.train = train
        self.path = '/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/'

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(self.path + row["path"]).convert("RGB")

        q = str(row["question"])
        a, b, c, d, hint, task = str(row["a"]), str(row["b"]), str(row["c"]), str(row["d"]), str(row['hint']).lstrip("[").lstrip("'").lstrip('"').rstrip("]").rstrip("'").rstrip('"'), row["task_hint"]
        user_text = build_mc_prompt(q, a, b, c, d, hint, task)

        # ✅ Qwen3는 멀티모달 메시지 기반이므로 구조 그대로 유지 가능
        messages = [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
            {"role": "user", "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": user_text}
            ]}
        ]
        if self.train:
            gold = str(row["answer"]).strip().lower()
            messages.append({"role": "assistant", "content": [{"type": "text", "text": gold}]})

        return {"messages": messages, "image": img}

# Online Image Augmentation
- 두 번째 Epoch 부터 online으로 augmentation이 적용되도록 Collator 설계

In [ ]:
from dataclasses import dataclass
from typing import Any, Optional
from torchvision import transforms
import torch

@dataclass
class DataCollator:
    processor: Any
    train: bool = True
    current_epoch: int = 0           # 현재 epoch (외부에서 업데이트)
    aug_prob: float = 0.25            # 각 transform의 기본 적용 확률

    def __post_init__(self):
        # 개별 transform 정의
        self.transforms_list = [
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomApply([transforms.ColorJitter(
                brightness=0.3)], p=self.aug_prob),
            transforms.RandomApply([transforms.RandomRotation(15)], p=self.aug_prob),
            transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=self.aug_prob),
        ]

        # 기본 변환 (항상 적용)
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((IMAGE_SIZE, IMAGE_SIZE))

    def __call__(self, batch):
        texts, images = [], []

        for sample in batch:
            messages = sample["messages"]
            img = sample["image"]

            text = self.processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(text)

            # 증강 적용 여부 결정
            if self.train and self.current_epoch > 0:
                # 각 transform을 확률적으로 적용
                transform_chain = transforms.Compose([
                    self.resize,
                    *[transforms.RandomApply([t], p=self.aug_prob) for t in self.transforms_list],
                    self.to_tensor
                ])
            else:
                # 첫 epoch 또는 평가 모드
                transform_chain = transforms.Compose([
                    self.resize,
                    self.to_tensor
                ])

            img = transform_chain(img)
            images.append(img)

        enc = self.processor(
            text=texts,
            images=images,
            padding=True,
            return_tensors="pt"
        )

        if self.train:
            enc["labels"] = enc["input_ids"].clone()

        return enc

# Data Split

In [ ]:
split = int(len(train_df)*0.95)
train_subset, valid_subset = train_df.iloc[:split], train_df.iloc[split:]

train_ds = VQAMCDataset(train_subset, processor, train=True)
valid_ds = VQAMCDataset(valid_subset, processor, train=True)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=DataCollator(processor, True))
valid_loader = DataLoader(valid_ds, batch_size=1, shuffle=False, collate_fn=DataCollator(processor, True))

print("✅ Qwen3 Dataset 준비 완료!")

✅ Qwen3 Dataset 준비 완료!


# 추론 결과 후처리 함수 작성

In [ ]:
def extract_choice(text: str) -> str:
    text = text.strip().lower()

    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a"
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last

    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a"

# 실험 관리 툴 wandb 연결

In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: baekus2209 (ssafy_A014) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Configuration 정리

In [ ]:
import math
import torch
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
# 2. WandB 초기화
wandb.init(
    project="SSAFY_AI_CHALLENGE",       # 팀 프로젝트 이름
    entity="ssafy_A014",       # 팀 엔터티 (팀 워크스페이스 이름)
    name="qwen3_8b_instruct_OCRhint_taskhint_r16_img448_5epoch",    # 실험 이름 (원하면 동적으로 변경 가능)
    # 실험 파라미터 설정
    config={
        "model": "Qwen3-VL-8B-instruct-LoRA",  # 모델 이름
        "epochs": 5,
        # "batch_size": 16,
        "grad_accum": 4,
        "lr": 1e-4,
        "scheduler": "linear_warmup_3%",
        "precision": "bfloat16",
        "img_size": 448,
        "scheduler": "cosine_warmup"
    }
)

# Train, Valid, Test 루프
- epoch 마다 submission 저장

In [ ]:
# 3. 설정
config = wandb.config
device = "cuda"

model = model.to(device)
GRAD_ACCUM = config.grad_accum
optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)

num_total_steps = config.epochs * math.ceil(len(train_loader)/GRAD_ACCUM)
num_warmup_steps = num_total_steps * 0.03
# num_training_steps = 1 * math.ceil(len(train_loader)/GRAD_ACCUM)
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps=num_warmup_steps,
                                            num_training_steps=num_total_steps)
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     int(num_training_steps*0.03),
#     num_training_steps
# )
scaler = torch.cuda.amp.GradScaler(enabled=True)

# 4. 학습 루프
global_step = 0

collator = DataCollator(processor, train=True)

for epoch in range(config.epochs):

    collator.current_epoch = epoch  # epoch 값 업데이트
    train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, collate_fn=collator)

    running = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [train]", unit="batch")
    for step, batch in enumerate(progress_bar, start=1):
        batch = {k:v.to(device) for k,v in batch.items()}

        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            outputs = model(**batch)
            loss = outputs.loss / GRAD_ACCUM

        scaler.scale(loss).backward()
        running += loss.item()

        if step % GRAD_ACCUM == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            global_step += 1

            avg_loss = running / GRAD_ACCUM
            progress_bar.set_postfix({"loss": f"{avg_loss:.3f}"})
            running = 0.0

            # WandB 로그 기록
            wandb.log({
                "train/loss": avg_loss,
                "train/learning_rate": scheduler.get_last_lr()[0],
                "global_step": global_step,
                "epoch": epoch + (step / len(train_loader))
            })

    # Validation
    model.eval()
    val_loss = 0.0
    val_steps = 0
    with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):
        for vstep, vb in enumerate(tqdm(valid_loader, desc=f"Epoch {epoch+1} [valid]", unit="batch"), start=1):
            vb = {k:v.to(device) for k,v in vb.items()}
            loss = model(**vb).loss.item()
            val_loss += loss
            val_steps += 1

            # Validation도 실시간으로 step별 기록
            wandb.log({
                "valid/loss": loss,
                "step": global_step + vstep,  # train과 같은 축에서 표시
                "epoch": epoch + (vstep / len(valid_loader))
            })

    avg_val_loss = val_loss / val_steps
    print(f"[Epoch {epoch+1}] valid loss {avg_val_loss:.4f}")

    # Validation 로그 기록
    wandb.log({
        "valid/loss": avg_val_loss,
        "epoch": epoch + 1
    })



    ##################################
    preds = []

    # 추론 루프
    for i in tqdm(range(len(test_df)), desc="Inference", unit="sample"):
        row = test_df.iloc[i]
        test_path = "/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/" + row["path"]
        img = Image.open(test_path).convert("RGB")
        user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"], str(row['hint']).lstrip("[").lstrip("'").lstrip('"').rstrip("]").rstrip("'").rstrip('"'), row["task_hint"])

        messages = [
            {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
            {"role":"user","content":[
                {"type":"image","image":img},
                {"type":"text","text":user_text}
            ]}
        ]

        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

        with torch.no_grad():
            out_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False,
                                    eos_token_id=processor.tokenizer.eos_token_id)
        output_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
        # print("output_text:", output_text)
        # print("extract_choice:", extract_choice(output_text))
        preds.append(extract_choice(output_text))

    # 제출 파일 생성
    submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
    submission.to_csv(f"/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/submission/submission_qwen3_vl_8b_OCRhint_taskhint_r16_img448_{epoch + 1}epoch.csv", index=False)
    print("Saved")






    model.train()


    # 5. 모델 저장 및 업로드
    SAVE_DIR = f"/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/qwen3_vl_8b_OCRhint_taskhint_r16_img448_lora_{epoch + 1}epoch"
    model.save_pretrained(SAVE_DIR)
    processor.save_pretrained(SAVE_DIR)
    print("Saved:", SAVE_DIR)

    # 모델 파일 WandB에 업로드 (선택)
    wandb.save(SAVE_DIR + "/*")
wandb.finish()


/tmp/ipython-input-2014728456.py:20: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)


Epoch 1 [train]:   0%|          | 0/3692 [00:00<?, ?batch/s]

/tmp/ipython-input-2014728456.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-2014728456.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(dtype=torch.bfloat16):


Epoch 1 [valid]:   0%|          | 0/195 [00:00<?, ?batch/s]

[Epoch 1] valid loss 3.0480


...

Inference:   0%|          | 0/3887 [00:00<?, ?sample/s]

Saved


wandb: WARNING Symlinked 12 files into the W&B run directory, call wandb.save again to sync new files.


Saved: /content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/qwen3_vl_8b_OCRhint_taskhint_r16_img448_lora_5epoch


epoch,▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▆▇▇█
global_step,▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█████
step,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▆▆▆▆▆▆▆████████
train/learning_rate,▆██████▇▇▇▇▇▇▇▇▆▆▆▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/loss,▄▃▅█▅▇▇▇▆▅▄▇▄▅▅▁▇█▆▆█▇▅▆▇█▆▄▇▄██▁▆█▄▆▆▂▇
valid/loss,▆█▃▇▇▇▇▁▇▇█▇▇▆▇▆▂▇▁▇▇█▇▇▅▆▅▂▇▇▇▇▇█▇▅▇█▇▇
epoch,5
global_step,4615
step,4810
train/learning_rate,0
train/loss,0.77545


# 세션 종료

In [ ]:
from google.colab import runtime
runtime.unassign()

# --------------